# SQL 

## Цель проекта

Проанализировать базу данных сервиса для чтения книг по подписке для формирования нового информационного продукта

### 1. Посчитайте, сколько книг вышло после 1 января 2000 года

In [9]:
SELECT COUNT(book_id)
FROM books
WHERE publication_date >='2000-01-01'

,count
0,821


### 2. Для каждой книги посчитайте количество обзоров и среднюю оценку

In [10]:
SELECT a.title,
       b.count_review,
       a.rating_mean
FROM
  (SELECT books.title,
          AVG(ratings.rating) AS rating_mean
   FROM ratings
   LEFT JOIN books ON ratings.book_id = books.book_id
   GROUP BY books.title) AS a
LEFT JOIN
  (SELECT books.title,
          COUNT(reviews.review_id) AS count_review
   FROM reviews
   LEFT JOIN books ON reviews.book_id = books.book_id
   GROUP BY books.title) AS b ON (a.title = b.title)

,title,count_review,rating_mean
0,The Count of Monte Cristo,5.0,4.217391
1,Count Zero (Sprawl #2),2.0,2.500000
2,The Botany of Desire: A Plant's-Eye View of th...,2.0,3.500000
3,The Poisonwood Bible,5.0,4.363636
4,The Canterbury Tales,3.0,3.333333
...,...,...,...
994,Of Love and Other Demons,2.0,4.500000
995,In the Heart of the Sea: The Tragedy of the Wh...,3.0,3.333333
996,Welcome to Temptation (Dempseys #1),2.0,5.000000
997,World's End (The Sandman #8),2.0,4.500000


### 3. Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры

In [15]:
SELECT publishers.publisher,
       COUNT(books.book_id) AS count_book
FROM publishers
LEFT JOIN books ON publishers.publisher_id = books.publisher_id
WHERE books.num_pages > 50
GROUP BY publishers.publisher
ORDER BY count_book DESC

,publisher,count_book
0,Penguin Books,42


### 4. Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками

In [16]:
SELECT a.author,
       AVG(book_50.avg_rating)
FROM
  (SELECT b.book_id,
          b.author_id,
          AVG(r.rating) AS avg_rating,
          COUNT(r.rating_id) AS count_user
   FROM books AS b
   LEFT JOIN ratings AS r ON b.book_id = r.book_id
   GROUP BY b.book_id
   HAVING COUNT(r.rating_id)>=50) AS book_50
JOIN authors AS a ON book_50.author_id = a.author_id
GROUP BY a.author
ORDER BY AVG(book_50.avg_rating) DESC

,author,avg
0,J.K. Rowling/Mary GrandPré,4.28


### 5. Посчитайте среднее количество обзоров от пользователей, которые поставили больше 48 оценок

In [17]:
SELECT AVG(b.reviews_count)
FROM
  (SELECT ratings.username,
          COUNT(ratings.rating_id) AS ratings_count
   FROM ratings
   GROUP BY ratings.username
   HAVING COUNT(ratings.rating_id) > 48) AS a
INNER JOIN
  (SELECT reviews.username,
          COUNT(reviews.review_id) AS reviews_count
   FROM reviews
   GROUP BY reviews.username) AS b ON (a.username = b.username)

,avg
0,24.0


### 6. Выведите таблицу, которая будет содержать по году публикации:
- количество издательств,
- выпущенных книг и
- сколько всего тысяч страниц было в изданных книгах

Отобразить нужно только те года, в которых издано более 30 книг.

In [18]:
SELECT EXTRACT(YEAR FROM publication_date) AS year,
       COUNT(DISTINCT publisher_id) as publisher_count,
       COUNT(book_id) as book_count,
       SUM(num_pages) / 1000 as sum_pages
   FROM books
   GROUP BY EXTRACT(YEAR FROM publication_date) 
   HAVING COUNT(book_id) > 30
   ORDER BY year DESC

,year,publisher_count,book_count,sum_pages
0,2007.0,38,50,18
1,2006.0,109,184,68
2,2005.0,89,139,55
3,2004.0,88,124,46
4,2003.0,65,105,41
5,2002.0,62,94,38
6,2001.0,41,60,21
7,2000.0,35,38,13
8,1999.0,26,41,15


### 7. Выведите в одной таблице два числа — среднюю оценку тех книг, на которые написало отзывов более 3 человек, и отдельно среднюю оценку остальных книг, а также сделай выводы какой рейтинг больше.

In [19]:
WITH over_3 AS
  (SELECT book_id,
          COUNT(DISTINCT username) AS count_user_over_3
   FROM reviews
   GROUP BY book_id
   HAVING COUNT(DISTINCT username) > 3),
   
     under_3 AS
  (SELECT book_id,
          COUNT(DISTINCT username) AS count_user_under_3
   FROM reviews
   GROUP BY book_id
   HAVING COUNT(DISTINCT username) <= 3)
   
SELECT ROUND(AVG(rating), 2) AS avg_ratings
FROM over_3
LEFT JOIN ratings ON over_3.book_id = ratings.book_id

UNION ALL

SELECT ROUND(AVG(rating), 2) AS avg_ratings
FROM under_3
LEFT JOIN ratings ON under_3.book_id = ratings.book_id

,avg_ratings
0,3.95
1,3.89
